In [1]:
import pandas as pd
import numpy as np

import datetime
print (str(datetime.datetime.today() ) )

print ("Import packages")
#pip install cx_Oracle
#pip install sqlalcheny
#Download Oracle- https://www.oracle.com/database/technologies/instant-client/downloads.html
#Connect to cx_Oracle and import packages
import cx_Oracle

#create path to downloaded oracle files and initiate
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_9")

import sqlalchemy
from sqlalchemy.schema import CreateIndex
from sqlalchemy import Table, Column, Integer, Numeric,String, ForeignKey
from sqlalchemy import MetaData
from sqlalchemy import insert
metadata= MetaData()

from sqlalchemy.engine import create_engine


print ("DONE")

2023-10-12 11:32:17.025072
Import packages
DONE


### Import Labor Transactions

In [4]:
path = ""


import pandas as pd

print ("Import labor transactions")

import time
start = time.time()

import datetime
print (str(datetime.datetime.today()).split()[0] )


#from datetime import datetime
#print (start )

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)


labtrans_query = """

select labtrans.LABORCODE,
       labtrans.ENTERDATE,
       labtrans.REFWO,
       labtrans.TRANSTYPE,
       labtrans.startdatetime,
       labtrans.finishdatetime,
       labtrans.location

       
from Maximo.labtrans labtrans

--WHERE TRUNC(labtrans.enterdate) >= sysdate - interval '1' year AND (labtrans.transtype = 'WORKWITHSEQ' OR labtrans.transtype = 'WORK')
WHERE labtrans.enterdate >= '01-JAN-2023'  AND (labtrans.transtype = 'WORKWITHSEQ' OR labtrans.transtype = 'WORK' OR labtrans.transtype = 'NOBLDGACCESS' OR labtrans.transtype = 'RESREFUSED' OR labtrans.transtype ='RESNOTHOME' OR labtrans.transtype ='UNFOUNDED' )

"""
#(labtrans.enterdate >= '01-JAN-2022')

#WHERE TRUNC(xrf_det.entrydate) >= sysdate - interval '1' year AND xrf_det.finalclassification = 'Positive' AND xrf_det.side != 'Calibrate' AND xrf_det.side != 'CALIBRATE'
#(labtrans.enterdate >= '12-JAN-2023')
#(labtrans.enterdate >= '01-JAN-2022' AND labtrans.enterdate <= '1-MAR-2022')



dl = pd.read_sql_query(labtrans_query, engine)

print (len(dl))

print ("DONE")

Import labor transactions
2023-10-12
2308419
DONE


In [5]:
#dl.to_csv( path + "Labor Transactions 1_1_23-10_12_23.csv", index=False)

In [6]:
dl2 = dl.copy()
dl1 = pd.read_csv(path + "Labor Transactions 2022.csv")

frames = [dl1, dl2]

dl = pd.concat(frames)

print (len(dl))

5160067


In [7]:
print (len(dl))

a = dl[dl["transtype"] == "UNFOUNDED"]
print ("unfounded", len(a))

a = dl[dl["transtype"] == "RESREFUSED"]
print ("res refused", len(a))

a = dl[dl["transtype"] == "RESNOTHOME"]
print ("res not home", len(a))

a = dl[dl["transtype"] == "NOBLDGACCESS"]
print ("no bldg access", len(a))


5160067
unfounded 359174
res refused 167133
res not home 555342
no bldg access 6244


### Import Work Orders to get wonum to subset labor trans and Insert Other Data

In [10]:

import datetime
print (str(datetime.datetime.today()) )


print ("Import work orders")
import time
start = time.time()


DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)

import pandas as pd


workorder_query = """

select max_wo.worktype,
       max_wo.problemcode,
       max_wo.failurecode,
       max_wo.wonum,
       max_wo.parent,
       TRUNC(max_wo.reportdate) AS report_date,
       max_wo.status,
       TRUNC(max_wo.statusdate) AS status_date,
       max_wo.location,
       max_wo.haschildren,
       max_wo.OWNERGROUP,
       max_wo.ACTSTART,
       max_wo.ACTFINISH,
       max_wo.ZZPESTSENSITIVE,
       max_wo.ZZCRAFT,
       max_wo.JPNUM
       
from Maximo.workorder max_wo

--WHERE TRUNC(max_wo.reportdate) >= sysdate - interval '1' year AND max_wo.status = 'CLOSE' AND max_wo.failurecode = 'EXTERMINATION' AND max_wo.worktype = 'CM' 

WHERE  max_wo.reportdate >= '01-JAN-2023' AND max_wo.status = 'CLOSE' AND max_wo.failurecode = 'EXTERMINATION' AND max_wo.worktype = 'CM' 


"""

#(max_wo.reportdate >= '01-JAN-2022')
#(max_wo.reportdate >= '01-JAN-2022' AND max_wo.reportdate <= '1-MAR-2022')

df = pd.read_sql_query(workorder_query, engine)

print (len(df))
df = df[pd.notnull(df['actstart'])]
print (len(df))


#print ("Subset for closed work orders")
#print (len(df))
#df = df[df["status"] == "CLOSE"]
#df = df[df["status"] == "APPR"]

#subsetlist = ["CLOSE", "APPR"]

#df = df[df['status'].isin(subsetlist)].reset_index(drop=True)

print (len(df))
df = df[df["jpnum"] == 'M10353']
df = df[df["zzcraft"] == 'EXTERMIN']
print (len(df))

print ("DONE")

#print (test_df)



2023-10-12 12:04:55.622167
Import work orders
230134
117354
117354
115026
DONE


In [11]:
#df.to_csv(path + "Work Orders 1_1_23-10_12_23.csv", index=False)

In [12]:
df2 = df.copy()
df1 = pd.read_csv(path + "Work Orders 2022.csv")

frames = [df1, df2]

df = pd.concat(frames)

print (len(df))

254880


Subset Labor Transactions

In [13]:
import datetime
print (str(datetime.datetime.today()) )

print ("Subset Labor Transactions")

df["wonum"] = df["wonum"].astype(str).map(str.strip)
dl['refwo'] = dl['refwo'].astype(str).map(str.strip)

df = df.reset_index(drop=True)
dflist = df["wonum"].tolist()

print (len(dl))

dl2 = dl[dl['refwo'].isin(dflist)].reset_index(drop=True)

print (len(dl2))

print ("DONE")



2023-10-12 12:10:05.363668
Subset Labor Transactions
5160067
347414
DONE


### Insert Work Order Fields into Labor Transactions

In [14]:
print (str(datetime.datetime.today()) )


dftypes = ["failurecode", "problemcode", "ownergroup", "zzcraft"]

for d in dftypes:
    dfDic = df.set_index('wonum')[d].to_dict()
    dl2[d] = dl2["refwo"].map(dfDic)

print ("DONE")

2023-10-12 12:10:28.351136
DONE


### Import Person Data and Merge with Labor Transactions

In [15]:
print ("Import person data")
print (str(datetime.datetime.today()) )


#from datetime import datetime
#print (start )

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)


person_query = """

select *  
       
from Maximo.person 

"""

#WHERE TRUNC(xrf_det.entrydate) >= sysdate - interval '1' year AND xrf_det.finalclassification = 'Positive' AND xrf_det.side != 'Calibrate' AND xrf_det.side != 'CALIBRATE'


dp = pd.read_sql_query(person_query, engine)
dp = dp[["personid", "status", "displayname","firstname","lastname","department","title", "zzhrtitle", "hiredate", "terminationdate"]]
dp["personid"] = dp["personid"].astype(str).map(str.strip)
dp = dp.rename(columns={'personid': 'laborcode'})

print ("DONE")

import datetime
print (str(datetime.datetime.today()) )


dl2["laborcode"] = dl2["laborcode"].astype(str).map(str.strip)
print (len(dl2))
dl2 = pd.merge(dl2, dp, how='left', on=['laborcode'])
print (len(dl2))

print ("DONE")

Import person data
2023-10-12 12:10:43.052673
DONE
2023-10-12 12:10:54.646647
347414
347414
DONE


In [9]:
#dl2.to_csv(path + "test.csv", index=False)

In [10]:
people = sorted(list(set(dl2["displayname"].tolist() ) ))
print (len(people))
dp2 = dp[dp['displayname'].isin(people)]



379


In [11]:
dp2.to_csv(path + "Exterminators from Person Dataset.csv", index=False)

In [12]:
transtype = set(dl2["transtype"].tolist() )
transtype

{'NOBLDGACCESS',
 'RESNOTHOME',
 'RESREFUSED',
 'UNFOUNDED',
 'WORK',
 'WORKWITHSEQ'}

In [8]:
a = dl2[dl2["transtype"] == "UNFOUNDED"]
print ("unfounded", len(a))

a = dl2[dl2["transtype"] == "RESREFUSED"]
print ("res refused", len(a))

a = dl2[dl2["transtype"] == "RESNOTHOME"]
print ("res not home", len(a))

a = dl2[dl2["transtype"] == "NOBLDGACCESS"]
print ("no bldg access", len(a))


unfounded 1607
res refused 33828
res not home 42537
no bldg access 1220


### Get IPM Inspection Questions

### With Labor Transactions Jan-Feb 2022

In [14]:
# wt = work orders table (workorder)
# iq = inspection questions (zzinsquestion)
# wr = work order results table (zzworesult)
# lt = labor transactions (labtrans)



import datetime
print (str(datetime.datetime.today()) )

print ("Import questions and results")
import time
start = time.time()

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)

import pandas as pd


ques_query = """

select wp.wonum wonum1, wp.description description1, wt.wonum wonum2, wt.description description2, 
iq.inslist, iq.questioncode, iq.description description3, wr.result, 
lt.laborcode,lt.ENTERDATE, lt.TRANSTYPE,lt.startdatetime,lt.finishdatetime

from workorder wp, workorder wt, zzinsquestion iq, zzworesult wr, labtrans lt

where  (wt.reportdate >= '01-JAN-2022' AND wt.reportdate <= '31-MAR-2022')

and wt.status = 'CLOSE' and wt.failurecode = 'EXTERMINATION' 
and wt.worktype = 'CM' 
and wp.wonum = wt.parent and wt.istask = 1
and wt.zzinslist = iq.inslist and nvl(wt.zzinslistrevnum, 0) = iq.revnum
and wr.wonum = wt.wonum and wr.siteid = wt.siteid and wr.questioncode = iq.questioncode
and lt.refwo = wp.wonum and lt.siteid = wp.siteid

"""



#(wt.reportdate >= '01-JAN-2022' AND wt.reportdate <= '1-MAR-2022')
#dq = pd.read_sql_query(ques_query, engine)

#path = "C:\\Users\\MehriD01\\Documents\\Pest\\"
#dq.to_csv(path + "Labor_Transactions_jan-mar_2022.csv", index=False)




2023-08-14 16:42:42.553119
Import questions and results


### April - June 2022

In [15]:
# wt = work orders table (workorder)
# iq = inspection questions (zzinsquestion)
# wr = work order results table (zzworesult)
# lt = labor transactions (labtrans)


import datetime
print (str(datetime.datetime.today()) )

print ("Import questions and results")
import time
start = time.time()

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)

import pandas as pd


ques_query = """

select wp.wonum wonum1, wp.description description1, wt.wonum wonum2, wt.description description2, 
iq.inslist, iq.questioncode, iq.description description3, wr.result, 
lt.laborcode,lt.ENTERDATE, lt.TRANSTYPE,lt.startdatetime,lt.finishdatetime

from workorder wp, workorder wt, zzinsquestion iq, zzworesult wr, labtrans lt

where  (wt.reportdate >= '01-APR-2022' AND wt.reportdate <= '30-JUN-2022')

and wt.status = 'CLOSE' and wt.failurecode = 'EXTERMINATION' 
and wt.worktype = 'CM' 
and wp.wonum = wt.parent and wt.istask = 1
and wt.zzinslist = iq.inslist and nvl(wt.zzinslistrevnum, 0) = iq.revnum
and wr.wonum = wt.wonum and wr.siteid = wt.siteid and wr.questioncode = iq.questioncode
and lt.refwo = wp.wonum and lt.siteid = wp.siteid


"""


#(wt.reportdate >= '01-JAN-2022' AND wt.reportdate <= '1-MAR-2022')
#dq = pd.read_sql_query(ques_query, engine)

#path = "C:\\Users\\MehriD01\\Documents\\Pest\\"
#dq.to_csv(path + "Labor_Transactions_apr-jun_2022.csv", index=False)


2023-08-14 16:42:43.762249
Import questions and results


### July - Sept 2022

In [16]:
# wt = work orders table (workorder)
# iq = inspection questions (zzinsquestion)
# wr = work order results table (zzworesult)
# lt = labor transactions (labtrans)


import datetime
print (str(datetime.datetime.today()) )

print ("Import questions and results")
import time
start = time.time()

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)

import pandas as pd


ques_query = """

select wp.wonum wonum1, wp.description description1, wt.wonum wonum2, wt.description description2, 
iq.inslist, iq.questioncode, iq.description description3, wr.result, 
lt.laborcode,lt.ENTERDATE, lt.TRANSTYPE,lt.startdatetime,lt.finishdatetime

from workorder wp, workorder wt, zzinsquestion iq, zzworesult wr, labtrans lt

where  (wt.reportdate >= '01-JUL-2022' AND wt.reportdate <= '30-SEP-2022')

and wt.status = 'CLOSE' and wt.failurecode = 'EXTERMINATION' 
and wt.worktype = 'CM' 
and wp.wonum = wt.parent and wt.istask = 1
and wt.zzinslist = iq.inslist and nvl(wt.zzinslistrevnum, 0) = iq.revnum
and wr.wonum = wt.wonum and wr.siteid = wt.siteid and wr.questioncode = iq.questioncode
and lt.refwo = wp.wonum and lt.siteid = wp.siteid


"""


#(wt.reportdate >= '01-JAN-2022' AND wt.reportdate <= '1-MAR-2022')
#dq = pd.read_sql_query(ques_query, engine)

#path = "C:\\Users\\MehriD01\\Documents\\Pest\\"
#dq.to_csv(path + "Labor_Transactions_jul-sept_2022.csv", index=False)

2023-08-14 16:42:44.517580
Import questions and results


### Oct - Dec 2022

In [17]:
# wt = work orders table (workorder)
# iq = inspection questions (zzinsquestion)
# wr = work order results table (zzworesult)
# lt = labor transactions (labtrans)


import datetime
print (str(datetime.datetime.today()) )

print ("Import questions and results")
import time
start = time.time()

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)

import pandas as pd


ques_query = """

select wp.wonum wonum1, wp.description description1, wt.wonum wonum2, wt.description description2, 
iq.inslist, iq.questioncode, iq.description description3, wr.result, 
lt.laborcode,lt.ENTERDATE, lt.TRANSTYPE,lt.startdatetime,lt.finishdatetime

from workorder wp, workorder wt, zzinsquestion iq, zzworesult wr, labtrans lt

where  (wt.reportdate >= '01-OCT-2022' AND wt.reportdate <= '31-DEC-2022')

and wt.status = 'CLOSE' and wt.failurecode = 'EXTERMINATION' 
and wt.worktype = 'CM' 
and wp.wonum = wt.parent and wt.istask = 1
and wt.zzinslist = iq.inslist and nvl(wt.zzinslistrevnum, 0) = iq.revnum
and wr.wonum = wt.wonum and wr.siteid = wt.siteid and wr.questioncode = iq.questioncode
and lt.refwo = wp.wonum and lt.siteid = wp.siteid


"""


#(wt.reportdate >= '01-JAN-2022' AND wt.reportdate <= '1-MAR-2022')
#dq = pd.read_sql_query(ques_query, engine)

#path = "C:\\Users\\MehriD01\\Documents\\Pest\\"
#dq.to_csv(path + "Labor_Transactions_oct-dec_2022.csv", index=False)

2023-08-14 16:42:45.499057
Import questions and results


### Jan 2023 - Present

In [24]:
# wt = work orders table (workorder)
# iq = inspection questions (zzinsquestion)
# wr = work order results table (zzworesult)
# lt = labor transactions (labtrans)


import datetime
print (str(datetime.datetime.today()) )

print ("Import questions and results")
import time
start = time.time()

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)

import pandas as pd


ques_query = """

select wp.wonum wonum1, wp.description description1, wt.wonum wonum2, wt.description description2, 
iq.inslist, iq.questioncode, iq.description description3, wr.result, 
lt.laborcode,lt.ENTERDATE, lt.TRANSTYPE,lt.startdatetime,lt.finishdatetime

from workorder wp, workorder wt, zzinsquestion iq, zzworesult wr, labtrans lt

where  wt.reportdate >= '31-MAR-2023'


and wt.status = 'CLOSE' and wt.failurecode = 'EXTERMINATION' 
and wt.worktype = 'CM' 
and wp.wonum = wt.parent and wt.istask = 1
and wt.zzinslist = iq.inslist and nvl(wt.zzinslistrevnum, 0) = iq.revnum
and wr.wonum = wt.wonum and wr.siteid = wt.siteid and wr.questioncode = iq.questioncode
and lt.refwo = wp.wonum and lt.siteid = wp.siteid


"""


#(wt.reportdate >= '01-JAN-2022' AND wt.reportdate <= '1-MAR-2022')
dq = pd.read_sql_query(ques_query, engine)

print (len(dq))

path = "C:\\Users\\MehriD01\\Documents\\Pest\\"
dq.to_csv(path + "Labor_Transactions_mar_2023-present.csv", index=False)

2023-10-12 12:35:25.564953
Import questions and results
3253521


### Concatinate Data

In [25]:
print (str(datetime.datetime.today()) )

#dq1 = pd.read_csv(path + "Labor_Transactions_jan-mar_2022.csv")
#dq2 = pd.read_csv(path + "Labor_Transactions_apr-jun_2022.csv")
#dq3 = pd.read_csv(path + "Labor_Transactions_jul-sept_2022.csv")
#dq4 = pd.read_csv(path + "Labor_Transactions_oct-dec_2022.csv")
#dq5 = pd.read_csv(path + "Labor_Transactions_jan-mar_2023.csv")
#dq6 = pd.read_csv(path + "Labor_Transactions_mar_2023-present.csv")

#print (dq1.shape)

#frames = [dq1, dq2, dq3, dq4, dq5, dq6]

dq2 = dq.copy()
dq1 = pd.read_csv(path + "Labor_Transactions_ALL_jan_22-present.csv")
frames = [dq1, dq2]


dq = pd.concat(frames)
dq = dq.reset_index(drop=True)

print (dq.shape)

print (len(dq))

dq = dq[["wonum1", "description3", "result"]]

print ("DONE")

2023-10-12 12:59:29.320511


C:\Users\MehriD01\AppData\Local\Temp\ipykernel_6536\2401751053.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dq4 = pd.read_csv(path + "Labor_Transactions_oct-dec_2022.csv")


(1172278, 13)
(9538360, 13)
9538360
DONE


In [26]:
#dq.to_csv(path + "Labor_Transactions_ALL_jan_22-present.csv", index=False)

In [ ]:
#dq.to_csv(path + "test.csv", index=False)

In [27]:

dq["wonum1"] = dq["wonum1"].astype(str).map(str.strip)
dl2['refwo'] = dl2['refwo'].astype(str).map(str.strip)

wolist = list(set(dq["wonum1"].tolist()  ))

print (len(dl2))
dl3 = dl2[dl2['refwo'].isin(wolist)].reset_index(drop=True)
print (len(dl3))

347414
265901


### Insert Qustions and Results into Labor Transactions

In [28]:
#desc = sorted(list(set(dq["description3"].tolist() ) ))
#desc.remove('Adjacent Apartment 1')
#desc.remove('Adjacent Apartment 2')
#desc.remove('Adjacent Apartment 3')
#desc.remove('Adjacent Apartment 4')


desc = ["Was Resident Educational Literature issued","Frass Removal", "HEPA Vacuum", 
       "Fill holes with excluder mesh or similar product","Sealing", "Escutcheon Plate Installation" ]

dq2 = dq.copy()

dq2["wonum1"] = dq2["wonum1"].astype(str).map(str.strip)
dl3['refwo'] = dl3['refwo'].astype(str).map(str.strip)


for d in desc:
    #print (d)
    dq3 = dq2[dq2["description3"] == d]
    print (d)
    print (len(dq3) )
    #print (dq3)
    
    dq3Dic = dq3.set_index('wonum1')['result'].to_dict()

    dl3[d] = dl3["refwo"].map(dq3Dic)



Was Resident Educational Literature issued
130670
Frass Removal
217833
HEPA Vacuum
217833
Fill holes with excluder mesh or similar product
273396
Sealing
273396
Escutcheon Plate Installation
210824


### Calculate Time

In [29]:
print (str(datetime.datetime.today()) )
print ("Calculate time")

dl3['startdatetime'] = pd.to_datetime(dl3['startdatetime'])

dl3['finishdatetime'] = pd.to_datetime(dl3['finishdatetime'])


#df2['Hours'] = (df2["actfinish"] - df2["actstart"]).dt.days

pd.set_option('chained_assignment', None)

dl3 = dl3.reset_index(drop=True)

dl3['Seconds'] = (dl3["finishdatetime"] - dl3["startdatetime"]).dt.seconds
dl3['Hours'] = (dl3["finishdatetime"] - dl3["startdatetime"]).dt.seconds / 3600.0
dl3['Minutes'] = (dl3["finishdatetime"] - dl3["startdatetime"]).dt.seconds / 60.0
dl3['Days'] = (dl3["finishdatetime"] - dl3["startdatetime"]).dt.days

dl3["Hours"] = dl3["Hours"].astype(float)
dl3["Hours"] = dl3["Hours"].round(2)

dl3["Minutes"] = dl3["Minutes"].astype(float)
dl3["Minutes"] = dl3["Minutes"].round(2)
print ("DONE")

2023-10-12 13:56:45.412471
Calculate time
DONE


### Merge Building and Development Data

In [30]:
import pandas as pd

print (str(datetime.datetime.today()) )

dl3['location'] = dl3['location'].astype(str)

dl3["TDS"] = dl3['location'].str.split(".").str[0]

print ("Import nycha_org_dim")

import time
start = time.time()

import datetime
print (str(datetime.datetime.today()).split()[0] )

#from datetime import datetime
#print (start )

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
#HOST = 'exadm1-scan' #enter the oracle db host url
HOST = '' #enter the oracle db host url

PORT = 1521 # enter the oracle port number
SERVICE = 'maxdb' # enter the oracle db service name
ENGINE_PATH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH)


nycha_org_query = """

select *
       
from maximo.nycha_org_dim nychaorg


"""

dnycha = pd.read_sql_query(nycha_org_query, engine)

subset = ['tds_num', 'borough_name', 'development_name']

dnycha= dnycha[subset]


tds = dnycha.tds_num.tolist()

tdslist = []

for c in tds:
    c = str(c)
    
    #print (c)
    
    if len(c) == 1:
        c = "00" + c
        tdslist.append(c)
    elif len(c) == 2:
        c = "0" + c
        tdslist.append(c)
    else:
        tdslist.append(c)


print (len(tdslist))
print (len(dnycha))

qn = pd.DataFrame(tdslist)

qn = qn.rename(columns={0: 'TDS'})

dnycha = pd.concat([dnycha, qn], axis=1)

print ("Merge with labor transactions")
print (len(dl3))
dl3 = pd.merge(dl3, dnycha, how='left', on=['TDS'])
print (len(dl3))


2023-10-12 13:56:53.254966
Import nycha_org_dim
2023-10-12
398
398
Merge with labor transactions
265901
265901


### Insert Address

In [31]:
print (str(datetime.datetime.today()).split()[0] )

path2 = "C:\\Users\\MehriD01\\Documents\\Data\\"

db = pd.read_csv(path2 + "zzbuildings.csv", low_memory=False)

db["BUILDING"] = db["BUILDING"].astype(str)
db["BLDGNO"] = db["BLDGNO"].astype(str)

db["TDS"] = db["BUILDING"].str.split(".").str[0]

#convert 1 to 01
for i in range(0, len(db)):
    a = db["BLDGNO"][i]
    if len(a) == 1:
        db["BLDGNO"][i] = "0" + db["BLDGNO"][i]
        
    b = db["TDS"][i]
    if len(b) == 1:
        db["TDS"][i] = "00" + db["TDS"][i]
    if len(b) == 2:
        db["TDS"][i] = "0" + db["TDS"][i] 
        


db["Building"] = db["TDS"] + "." + db["BLDGNO"]



sitid = set(db["SITEID"].tolist() )
print(sitid)

db["SITEID"] = db["SITEID"].str.replace('BK', 'BROOKYN')
db["SITEID"] = db["SITEID"].str.replace('BX', 'BRONX')
db["SITEID"] = db["SITEID"].str.replace('QS', 'QUEENS/STATEN ISLAND')
db["SITEID"] = db["SITEID"].str.replace('MN', 'MANHATTAN')

db["ADDRESS"] = db["ADDRESS"] + ", " + db["SITEID"]


dl3["Building No"] = dl3['location'].str.split(".").str[1]

dl3["TDS"] = dl3["TDS"].astype(str)
dl3["Building"] = dl3["TDS"] + "." + dl3["Building No"]


#dictionary to insert data
dbDic = db.set_index('Building')['ADDRESS'].to_dict()

dl3["Building Address"] = dl3["Building"].map(dbDic)

dl3["Building Address"] = dl3["Building Address"].astype(str).map(str.strip)

print ("DONE")

2023-10-12
{'BK', 'QS', 'MN', 'BX'}
DONE


### Insert Lat/Lon

In [32]:
print (str(datetime.datetime.today()).split()[0] )

path2 = "C:\\Users\\MehriD01\\Documents\\Data\\"

dgis = pd.read_csv(path2 + "nycha_loc_gps.csv", low_memory=False)

dgis["TDS_NUM"] = dgis["TDS_NUM"].astype(str)
dgis["BLDG_NUM"] = dgis["BLDG_NUM"].astype(str)

for i in range(0, len(dgis)):
    a = dgis["BLDG_NUM"][i]
    if len(a) == 1:
        dgis["BLDG_NUM"][i] = "0" + dgis["BLDG_NUM"][i]
        
    b = dgis["TDS_NUM"][i]
    if len(b) == 1:
        dgis["TDS_NUM"][i] = "00" + dgis["TDS_NUM"][i]
    if len(b) == 2:
        dgis["TDS_NUM"][i] = "0" + dgis["TDS_NUM"][i]            

dgis["Building"] = dgis["TDS_NUM"] + "." + dgis["BLDG_NUM"]

lat = dgis.set_index('Building')['LATITUDE'].to_dict()
dl3["lat"] = dl3["Building"].map(lat)

lon = dgis.set_index('Building')['LONGITUDE'].to_dict()
dl3["lon"] = dl3["Building"].map(lon)

print ("DONE")


2023-10-12
DONE


### Flag Work orders < 10 Minutes > 120 Minutes

In [33]:
print (str(datetime.datetime.today()).split()[0] )

minutes = dl3["Minutes"].tolist()
minutes

mintues30flag = []
mintues10flag = []
mintues120flag = []
greater10minutesflag = []


for i in range(0, len(minutes)):
    if minutes[i] < 30:
        mintues30flag.append(1)
    else:
        mintues30flag.append(0)
        
        
    if minutes[i] < 10:
        mintues10flag.append(1)
    else:
        mintues10flag.append(0)
        
        
    if minutes[i] > 120:
        mintues120flag.append(1)
    else:
        mintues120flag.append(0)
        
    if minutes[i] > 10:
        greater10minutesflag.append("Yes")
    else:
        greater10minutesflag.append("No")
        
        
        
print (len(mintues30flag))
print (len(mintues10flag))
print (len(mintues120flag))
print (len(greater10minutesflag))
print (len(dl3))

dl3["<30 min"] = mintues30flag
dl3["<10 min"] = mintues10flag
dl3[">120 min"] = mintues120flag
dl3[">10 min"] = greater10minutesflag

print ("DONE")

2023-10-12
265901
265901
265901
265901
265901
DONE


### Create Apartment String

In [34]:
dl3 = dl3.reset_index(drop=True)

#line = "071.04.008.F01.01A.KIT01"
#updated_line = '.'.join(line.split('.')[:-1])
print (str(datetime.datetime.today()).split()[0] )


updated_line = []

for i in range(0, len(dl3)):
    line = dl3["location"][i]
    linesplit = line.split(".")
    
    if len(linesplit) == 6:
        a = '.'.join(line.split('.')[:-1])
        #print (a)
        updated_line.append(a)
    else:
        #print (line)
        updated_line.append(line)

print (len(dl3))
print (len(updated_line))

dl3["Apartment"] = updated_line

2023-10-12
265901
265901


### Calculate occurence of questions

In [35]:
#dl3["Percent No"] = ""
print (str(datetime.datetime.today()).split()[0] )

dl3["Was Resident Educational Literature issued"] = dl3["Was Resident Educational Literature issued"].astype(str)
dl3["Frass Removal"] = dl3["Frass Removal"].astype(str)
dl3["HEPA Vacuum"] = dl3["HEPA Vacuum"].astype(str)
dl3["Fill holes with excluder mesh or similar product"] = dl3["Fill holes with excluder mesh or similar product"].astype(str)
dl3["Sealing"] = dl3["Sealing"].astype(str)
dl3["Escutcheon Plate Installation"] = dl3["Escutcheon Plate Installation"].astype(str)


percent_no = []
percent_yes = []
percent_nan = []

for i in range(0, len(dl3)):
    count = []
    d = dl3["Was Resident Educational Literature issued"][i]
    count.append(d)
    
    d = dl3["Frass Removal"][i]
    count.append(d)
    
    d = dl3["HEPA Vacuum"][i]
    count.append(d)
    
    d = dl3["Fill holes with excluder mesh or similar product"][i]
    count.append(d)
    
    d = dl3["Sealing"][i]
    count.append(d)
    
    d = dl3["Escutcheon Plate Installation"][i]
    count.append(d)
    
    n = (count.count('N')/6)
    n = round(n, 2)
    y = (count.count('Y')/6)
    y = round(y, 2)
    nan = (count.count('nan')/6)
    nan = round(nan, 2)

    percent_no.append(n)
    percent_yes.append(y)
    percent_nan.append(nan)

    

print (len(dl3))   
print (len(percent_no))   
print (len(percent_yes))   
print (len(percent_nan))   

 
dl3["Percent No"] = percent_no
dl3["Percent Yes"] = percent_yes
dl3["Percent nan"] = percent_nan


2023-10-12
265901
265901
265901
265901


### Convert Transaction Type to Dummies

In [36]:
print (str(datetime.datetime.today()).split()[0] )

print ("Convert transactions to dummies")
transaction_dummies = pd.get_dummies(dl3["transtype"])

transaction_dummies["WORK"] = transaction_dummies["WORK"] + transaction_dummies["WORKWITHSEQ"]

subset =  ['NOBLDGACCESS', 'RESNOTHOME', 'RESREFUSED', 'UNFOUNDED', 'WORK']
transaction_dummies = transaction_dummies[subset]

transaction_dummies.to_csv ( path + "test.csv", index=False)

print (len(transaction_dummies))
print (len(dl3))

dl4 = pd.concat([dl3, transaction_dummies], axis=1)

print (len(dl4))

print ("DONE")

2023-10-12
Convert transactions to dummies
265901
265901
265901
DONE


In [35]:
cols = transaction_dummies.columns
cols

Index(['NOBLDGACCESS', 'RESNOTHOME', 'RESREFUSED', 'UNFOUNDED', 'WORK'], dtype='object')

### Remove Non-NYCHA Employees

In [37]:
print (str(datetime.datetime.today()).split()[0] )


print ("remove non nycha employees")

dn = pd.read_csv(path + "7A Exterminators Group_Validation_Records as of 03-30-2023.csv")

dn["Badge #"] = dn["Badge #"].astype(str).map(str.strip)

badge = dn["Badge #"].tolist()


for i in range(len(badge)):
    #print (len(badge[i]))
    
    if len(badge[i]) == 2:
        badge[i] = "0000" + badge[i]
        
    if len(badge[i]) == 3:
        badge[i] = "000" + badge[i]
        
    if len(badge[i]) == 4:
        badge[i] = "00" + badge[i]
        
    if len(badge[i]) == 5:
        badge[i] = "0" + badge[i]
        
dn["Badge #"] = badge

dn["Employee Type"] = "NYCHA Employee"

dnDic = dn.set_index('Badge #')['Employee Type'].to_dict()


dl4["laborcode"] = dl4["laborcode"].astype(str).map(str.strip)

dl4["Employee Type"] = dl4["laborcode"].map(dnDic)

dl4["Employee Type"] = dl4["Employee Type"].fillna("Other")

print ("len before drop", len(dl4))

dl4 = dl4[dl4["Employee Type"] == "NYCHA Employee"]

print ("len after drop", len(dl4))

print ("DONE")


dl4 = dl4.reset_index(drop=True)


2023-10-12
remove non nycha employees
len before drop 265901
len after drop 195072
DONE


### Change Field Names

In [38]:
dl4["Finish Date"] = dl4["finishdatetime"].apply(lambda x: x.date())
dl4["Start Date"] = dl4["startdatetime"].apply(lambda x: x.date())
dl4 = dl4.rename(columns={'problemcode': 'Problem Code'})
dl4 = dl4.rename(columns={'failurecode': 'Failure Code'})
dl4 = dl4.rename(columns={'ownergroup': 'Owner Group'})
dl4 = dl4.rename(columns={'transtype': 'Transaction Type'})
dl4 = dl4.rename(columns={'status': 'Status'})
dl4 = dl4.rename(columns={'displayname': 'Name'})
dl4 = dl4.rename(columns={'department': 'Department'})
dl4 = dl4.rename(columns={'title': 'Title'})
dl4 = dl4.rename(columns={'zzhrtitle': 'HR Title'})
dl4 = dl4.rename(columns={'zzcraft': 'Craft'})
dl4 = dl4.rename(columns={'Enterdate': 'Enter Date'})


dl4["Count"] = 1

### Calculate Mode

In [ ]:
#transtype = list(set(dl4["Transaction Type"].tolist() ) )
#transtype
#transtype = list(set(dl4["Transaction Type 2"].tolist() ) )
#transtype


In [39]:
print (str(datetime.datetime.today()).split()[0] )

#dl5 = dl4["Name", "Transaction Type", "Minutes"]



dl4["Transaction Type 2"] = dl4["Transaction Type"]
dl4["Transaction Type 2"] = dl4["Transaction Type 2"].str.replace('WORKWITHSEQ', 'WORK')
dl4["Transaction Type 2"] = dl4["Transaction Type 2"].str.replace('NOBLDGACCESS', 'NO ACCESS')
dl4["Transaction Type 2"] = dl4["Transaction Type 2"].str.replace('RESNOTHOME', 'NO ACCESS')
dl4["Transaction Type 2"] = dl4["Transaction Type 2"].str.replace('RESREFUSED', 'NO ACCESS')

transtype = list(set(dl4["Transaction Type 2"].tolist() ) )


print ("WORK")

qn = pd.DataFrame(columns=('Emplpoyee Name', 'Mode', 'Count of Mode', 'Total Number of Labor Transactions', 
                           'Transaction Type'))

names = sorted(list(set(dl4["Name"].tolist()  )))

dl4["Minutes"] = dl4["Minutes"].astype(int)
dl4["MinutesCat"] = dl4["Minutes"].astype(str)

for i in range(0, len(names)):
    n = names[i]
    
  
    dl5 = dl4[dl4["Name"] == n].reset_index(drop=True)
    dl5 = dl5[dl5["Transaction Type 2"] == "WORK"].reset_index(drop=True)

    #dfb2 = dfb2[dfb2["AptWatTemp"] != "nan_nan"].reset_index(drop=True)

    dl5["Count"] = 1

    dl5G = dl5[["MinutesCat", "Count"]]

    dl5G = dl5G.groupby(['MinutesCat']).sum()
    dl5G = dl5G.add_suffix('').reset_index()
    dl5G = dl5G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
        
    if len(dl5G) > 0:
        qn.loc[i] = [n,dl5G["MinutesCat"][0],dl5G["Count"][0], len(dl5), "WORK" ]

qn["Mode Relative Frequency"] = qn["Count of Mode"]/qn["Total Number of Labor Transactions"]
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].astype(float)
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].round(2)

qn1 = qn.copy()



print ("NO ACCESS")

qn = pd.DataFrame(columns=('Emplpoyee Name', 'Mode', 'Count of Mode', 'Total Number of Labor Transactions', 
                           'Transaction Type'))

names = sorted(list(set(dl4["Name"].tolist()  )))

dl4["Minutes"] = dl4["Minutes"].astype(int)
dl4["MinutesCat"] = dl4["Minutes"].astype(str)

for i in range(0, len(names)):
    n = names[i]
    
  
    dl5 = dl4[dl4["Name"] == n].reset_index(drop=True)
    dl5 = dl5[dl5["Transaction Type 2"] == "NO ACCESS"].reset_index(drop=True)

    #dfb2 = dfb2[dfb2["AptWatTemp"] != "nan_nan"].reset_index(drop=True)

    dl5["Count"] = 1

    dl5G = dl5[["MinutesCat", "Count"]]

    dl5G = dl5G.groupby(['MinutesCat']).sum()
    dl5G = dl5G.add_suffix('').reset_index()
    dl5G = dl5G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
        
    if len(dl5G) > 0:
        qn.loc[i] = [n,dl5G["MinutesCat"][0],dl5G["Count"][0], len(dl5), "NO ACCESS" ]

qn["Mode Relative Frequency"] = qn["Count of Mode"]/qn["Total Number of Labor Transactions"]
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].astype(float)
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].round(2)

qn2 = qn.copy()



print ("UNFOUNDED")

qn = pd.DataFrame(columns=('Emplpoyee Name', 'Mode', 'Count of Mode', 'Total Number of Labor Transactions', 
                           'Transaction Type'))

names = sorted(list(set(dl4["Name"].tolist()  )))

dl4["Minutes"] = dl4["Minutes"].astype(int)
dl4["MinutesCat"] = dl4["Minutes"].astype(str)

for i in range(0, len(names)):
    n = names[i]
    
  
    dl5 = dl4[dl4["Name"] == n].reset_index(drop=True)
    dl5 = dl5[dl5["Transaction Type 2"] == "NO ACCESS"].reset_index(drop=True)

    #dfb2 = dfb2[dfb2["AptWatTemp"] != "nan_nan"].reset_index(drop=True)

    dl5["Count"] = 1

    dl5G = dl5[["MinutesCat", "Count"]]

    dl5G = dl5G.groupby(['MinutesCat']).sum()
    dl5G = dl5G.add_suffix('').reset_index()
    dl5G = dl5G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
        
    if len(dl5G) > 0:
        qn.loc[i] = [n,dl5G["MinutesCat"][0],dl5G["Count"][0], len(dl5), "UNFOUNDED" ]

qn["Mode Relative Frequency"] = qn["Count of Mode"]/qn["Total Number of Labor Transactions"]
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].astype(float)
qn["Mode Relative Frequency"] = qn["Mode Relative Frequency"].round(2)

qn3 = qn.copy()


frames = [qn1, qn2, qn3]
result = pd.concat(frames).reset_index(drop=True)

print ("DONE")

2023-10-12
WORK
NO ACCESS
UNFOUNDED
DONE


### Calculate Number of People work On a Job

In [40]:
print (str(datetime.datetime.today()).split()[0] )


print ("Calculate number of people")

dol = dl4[["location", "Finish Date", "Name", "startdatetime", "finishdatetime", "Transaction Type" ] ]

dol = dol.sort_values(by = 'location', ascending=True).reset_index(drop=True)

dol["Finish Date"] = dol["Finish Date"].astype(str).map(str.strip)
dol["LocationDate"] = dol["location"] + " " + dol["Finish Date"]

locations = sorted(set(dol["location"].tolist() ))

#print (len(dol))
#dol = dol.drop_duplicates(["location", "Finish Date", "Name"]).reset_index(drop=True)
#print (len(dol))

numberList = []
nameList = []

for i in range(0, len(locations)):
#for i in range(0, 1000):

    l = locations[i]
    
    #print (l)
    #print (l)
    dol2 = dol[dol["location"] == l]
    
    dol2 = dol2.sort_values(by = 'Finish Date', ascending=True).reset_index(drop=True)
    
    #print (dol2)
    #print("******************")
    
    
    d = sorted(set(dol2["Finish Date"].tolist() ))
    
    #print (len(d))
    
    for j in range(0, len(d)):
        dol3 = dol2[dol2["Finish Date"] == d[j]].reset_index(drop=True)
        
        names = sorted(set(dol3["Name"].tolist() ))
        
        
        if (len(dol3) > 1 and len(names)>1):
            #print (dol3)
            #print("******************")
            #numberDic = {dol3["LocationDate"][0]}
            #numberDic[dol3["LocationDate"][0]] = len(names)
            numberList.append([dol3["LocationDate"][0], len(names)])
            nameList.append([dol3["LocationDate"][0], names])


    #print(dol2)
    #print("******************")
    
print (len(numberList))
print (len(nameList))
print ("DONE")

2023-10-12
Calculate number of people
21047
21047
DONE


Create Dictionaries and Combine with Labor Transactions

In [41]:
print ("Insert number on the job")
numberDict = {}
for l2 in numberList:
    numberDict[l2[0]] = l2[1]
    
nameDict = {}
for l2 in nameList:
    nameDict[l2[0]] = l2[1]
    
dl4["Finish Date"] = dl4["Finish Date"].astype(str).map(str.strip)
dl4["LocationDate"] = dl4["location"] + " " + dl4["Finish Date"]

dl4["Number on Job"] = dl4["LocationDate"].map(numberDict)
dl4["Employees on Job > 1"] = dl4["LocationDate"].map(nameDict)

#dl4 = dl4.drop('Employees on Job', 1)

dl4["Number on Job"] = dl4["Number on Job"].fillna(1)

print ("DONE")

Insert number on the job
DONE


In [ ]:
#cols = dl4.columns
#cols = sorted(cols)
#cols

In [ ]:
#dol.to_csv(path + "test_num_people.csv", index=False)

### Calculate Work Time per Day

In [42]:
path2 = "C:\\Users\\MehriD01\\Documents\\Pest\\data_dump\\"

import pandas as pd


dl4["Start Date"] = dl4["startdatetime"].apply(lambda x: x.date())
dl4["Start Time"] = dl4["startdatetime"]
dl4["Finish Time"] = dl4["finishdatetime"]

dl5 = dl4[[ "Name", "Start Date", "Finish Date", "Start Time", "Finish Time", "Minutes"]]

dl5 = dl5.sort_values(by = 'Start Date', ascending=True).reset_index(drop=True)

dl5["Start Date"] = dl5["Start Date"].astype(str).map(str.strip)
dl5["Finish Date"] = dl5["Finish Date"].astype(str).map(str.strip)


dates = sorted(list(set(dl5["Start Date"].tolist() )))

len(dates)



dt = pd.DataFrame(columns=('Name', 'Start Date', 'Finish Date', 'Day Start Time', 'Day End Time', 'Number of Jobs', 'Minutes Worked'))

all_data = pd.DataFrame()

for i in range(0, len(dates)):
#for i in range(0, 1):

    #print(i)
    dl6 = dl5[dl5["Start Date"] == dates[i]]
    dl6 = dl6[dl6["Finish Date"] == dates[i]]
    dl6 = dl6.reset_index(drop=True)
    
    names = list(set(dl6["Name"].tolist() ))
    
    for j in range(0, len(names)):
    #for j in range(0, 1):
    
        dl7 = dl6[dl6["Name"] == names[j]]
        
        totalMinWorked = dl7["Minutes"].sum()
        
        starttime = sorted(dl7["Start Time"].tolist() )
        endtime = sorted(dl7["Finish Time"].tolist(),  reverse=True )


        start = starttime[0]
        end = endtime[0]
        
        #print (start)
        #print (end)
        
        dt.loc[j] = [names[j],dates[i],dates[i], start, end, len(dl7), totalMinWorked]
        
        filename = "work_time" + str(i) + ".csv"
        
        dt.to_csv(path2 + filename )
        
        



    #all_data = all_data.append(dt,ignore_index=False).reset_index(drop=True)
    #all_data = all_data.concat(dt,ignore_index=False).reset_index(drop=True)


#all_data = all_data.sort_values(by = 'Start Date', ascending=True).reset_index(drop=True)


#all_data['Hours Worked'] = (all_data['Day End Time'] - all_data['Day Start Time']).dt.total_seconds()/3600
#all_data['Minutes Worked'] = (all_data['Day End Time'] - all_data['Day Start Time']).dt.total_seconds()/60

#all_data['Day Start Time'] = all_data['Day Start Time'].apply( lambda d : d.time() )

#all_data['Day End Time'] = all_data['Day End Time'].apply( lambda d : d.time() )

#all_data["Hours per Job"] = all_data["Hours Worked"]/all_data["Number of Jobs"]
#all_data["Minutes per Job"] = all_data["Minutes Worked"]/all_data["Number of Jobs"]
print ("Done")


Done


Read the Data back in

In [43]:
import pandas as pd
import glob
import os


print ("Read files back into dataframe") 

# Get a list of all CSV file paths in a directory
csv_files = glob.glob( 'C:\\Users\\MehriD01\\Documents\\Pest\\data_dump\\*.csv')

# Create an empty list to store DataFrames
dataframes = []

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    dt = pd.read_csv(file)
    dataframes.append(dt)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dataframes, ignore_index=True)

all_data = final_df.copy()

subset = ['Name', 'Start Date', 'Finish Date', 'Day Start Time','Day End Time', 'Number of Jobs', 'Minutes Worked']

all_data = all_data[subset]


# Get a list of all CSV file paths in the directory
csv_files = glob.glob(os.path.join(path2, '*.csv'))

# Loop through each CSV file and delete it
for file_path in csv_files:
    os.remove(file_path)

print("CSV files deleted successfully.")



print ("Done")






Read files back into dataframe
CSV files deleted successfully.
Done


"\n# Now final_df contains the combined data from all CSV files\nIn this example, you'll need to replace /path/to/csv/files/ with the actual path to the directory containing your CSV files. The glob.glob() function is used to retrieve a list of file paths matching a certain pattern (in this case, all .csv files in the specified directory).\n\nInside the loop, each CSV file is read into a DataFrame using pd.read_csv(file), and the resulting DataFrames are appended to the dataframes list. After the loop, the pd.concat() function is used to concatenate all the individual DataFrames into a single DataFrame named final_df.\n\nAlternatively, you can achieve the same result using a list comprehension:\n\npython\nCopy code\nimport pandas as pd\nimport glob\n\n# Get a list of all CSV file paths in a directory\ncsv_files = glob.glob('/path/to/csv/files/*.csv')\n\n# Read all CSV files into DataFrames and store them in a list comprehension\ndataframes = [pd.read_csv(file) for file in csv_files]\n\

In [44]:
all_data["Hours Worked"] = all_data["Minutes Worked"]/60.0
all_data["Hours per Job"] = all_data["Hours Worked"]/all_data["Number of Jobs"]
all_data["Minutes per Job"] = all_data["Minutes Worked"]/all_data["Number of Jobs"]


"""
all_data['Hours Worked'] = ""
#all_data['Minutes Worked'] = ""

for i in range(0, len(all_data)):
    d = all_data["Day End Time"][i] - all_data["Day Start Time"][i]

    diff = d.seconds

    diff = int(diff)
    
    all_data['Hours Worked'][i] = diff/3600.0
    #all_data['Minutes Worked'][i] = diff/60.0


    #print (diff)
    
all_data = all_data.sort_values(by = 'Hours Worked', ascending=False).reset_index(drop=True)

all_data["Hours per Job"] = all_data["Hours Worked"]/all_data["Number of Jobs"]
all_data["Minutes per Job"] = all_data["Minutes Worked"]/all_data["Number of Jobs"]
"""

'\nall_data[\'Hours Worked\'] = ""\n#all_data[\'Minutes Worked\'] = ""\n\nfor i in range(0, len(all_data)):\n    d = all_data["Day End Time"][i] - all_data["Day Start Time"][i]\n\n    diff = d.seconds\n\n    diff = int(diff)\n    \n    all_data[\'Hours Worked\'][i] = diff/3600.0\n    #all_data[\'Minutes Worked\'][i] = diff/60.0\n\n\n    #print (diff)\n    \nall_data = all_data.sort_values(by = \'Hours Worked\', ascending=False).reset_index(drop=True)\n\nall_data["Hours per Job"] = all_data["Hours Worked"]/all_data["Number of Jobs"]\nall_data["Minutes per Job"] = all_data["Minutes Worked"]/all_data["Number of Jobs"]\n'

In [45]:
all_data2 = all_data.drop_duplicates(subset=['Name', 'Day Start Time', 'Day End Time', 'Number of Jobs']).reset_index(drop=True)
print (len(all_data))
print (len(all_data2))


46385
31105


In [ ]:
cols = dl4.columns
cols = sorted(cols)


In [ ]:
all_data2.dtypes

Outside Normal Work Hours

In [46]:
#all_data2["Day End Time"][0].hour

all_data2['Day Start Time'] = pd.to_datetime(all_data2['Day Start Time'])
all_data2['Day End Time'] = pd.to_datetime(all_data2['Day End Time'])

all_data2["Outside Normal Work Hours"] = "No"

for i in range(0, len(all_data2)):
    if (all_data2["Day Start Time"][i].hour <= 7 or all_data2["Day End Time"][i].hour >= 21):
        all_data2["Outside Normal Work Hours"][i] = "Yes"
        
        
print ("DONE")

DONE


### Mode: Start and End Time

In [47]:
all_data2["Day Start Time"][0].hour

all_data2["Start Hour"] = ""
all_data2["End Hour"] = ""

for i in range(0, len(all_data2)):
    all_data2["Start Hour"][i] =  all_data2["Day Start Time"][i].hour
    all_data2["End Hour"][i] =  all_data2["Day End Time"][i].hour

print ("DONE")

DONE


In [48]:
names = sorted(list(set(all_data2["Name"].tolist()  )))

qnhours = pd.DataFrame(columns=('Emplpoyee Name', 'Mode Start Time', 'Count of Start Mode', 
                                'Mode End Time', 'Count of End Mode', 'Total Number of Days Worked'))


#dl4["Minutes"] = dl4["Minutes"].astype(int)
all_data2["Start Hour Cat"] = all_data2["Start Hour"].astype(str)
all_data2["End Hour Cat"] = all_data2["End Hour"].astype(str)


for i in range(0, len(names)):
    n = names[i]
    
  
    all_data3 = all_data2[all_data2["Name"] == n].reset_index(drop=True)
    all_data3["Count"] = 1
    
    #start time mode
    all_data3G = all_data3[["Start Hour Cat", "Count"]]
    all_data3G = all_data3G.groupby(['Start Hour Cat']).sum()
    all_data3G = all_data3G.add_suffix('').reset_index()
    all_data3G = all_data3G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
    starthourmode = all_data3G["Start Hour Cat"][0]
    starthourmodecount = all_data3G["Count"][0]
    
    #end time mode
    all_data3G = all_data3[["End Hour Cat", "Count"]]
    all_data3G = all_data3G.groupby(['End Hour Cat']).sum()
    all_data3G = all_data3G.add_suffix('').reset_index()
    all_data3G = all_data3G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
    endhourmode = all_data3G["End Hour Cat"][0]
    endhourmodecount = all_data3G["Count"][0]
    
        
    if len(all_data3) > 0:
        qnhours.loc[i] = [n,starthourmode, starthourmodecount,endhourmode,endhourmodecount, len(all_data3)]

qnhours["Start Mode Relative Frequency"] = qnhours["Count of Start Mode"]/qnhours["Total Number of Days Worked"]
qnhours["Start Mode Relative Frequency"] = qnhours["Start Mode Relative Frequency"].astype(float)
qnhours["Start Mode Relative Frequency"] = qnhours["Start Mode Relative Frequency"].round(2)

qnhours["End Mode Relative Frequency"] = qnhours["Count of End Mode"]/qnhours["Total Number of Days Worked"]
qnhours["End Mode Relative Frequency"] = qnhours["End Mode Relative Frequency"].astype(float)
qnhours["End Mode Relative Frequency"] = qnhours["End Mode Relative Frequency"].round(2)

#qn1 = qn.copy()

### Calculate the Days of the Week

In [49]:
# 0 = monday, 1=tuesday, 2=wednesday, 3=thursday, 4=friday, 5=saturday, 6=sunday
dl4["Day"] = dl4['Finish Time'].dt.dayofweek
dl4["Day"] = dl4["Day"].astype(str).map(str.strip)
dl4["Day"] = dl4["Day"].str.replace('0', 'Monday')
dl4["Day"] = dl4["Day"].str.replace('1', 'Tuesday')
dl4["Day"] = dl4["Day"].str.replace('2', 'Wednesday')
dl4["Day"] = dl4["Day"].str.replace('3', 'Thursday')
dl4["Day"] = dl4["Day"].str.replace('4', 'Friday')
dl4["Day"] = dl4["Day"].str.replace('5', 'Saturday')
dl4["Day"] = dl4["Day"].str.replace('6', 'Sunday')



In [50]:
dl4 = dl4.reset_index(drop=True)

dl4['Finish Time 2'] = dl4["Finish Time"].apply( lambda d : d.time() )
dl4['Finish Time 2'] = dl4['Finish Time 2'].astype(str).map(str.strip)
dl4['Finish Time 2'] = dl4['Finish Time 2'].str.split(":").str[0]
dl4['Finish Time 2'] = dl4['Finish Time 2'].astype(int)

dl4["Work After 4pm"] = "No"

for i in range(0, len(dl4)):
    if dl4['Finish Time 2'][i] >= 16:
        dl4["Work After 4pm"][i] = "Yes"


#if dl4["Finish Time 2"][0] > pd.datetime('13:10:00'):
#    print ("hey")

print ("DONE")

DONE


### Create Dummes for IPM Questions

In [51]:
print ("Create dummies")

a = dl4["Was Resident Educational Literature issued"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["Education Literature"] = dummies       


a = dl4["Frass Removal"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["Frass"] = dummies 


a = dl4["HEPA Vacuum"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["HEPA"] = dummies


a = dl4["Fill holes with excluder mesh or similar product"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["Exluder mesh"] = dummies       


a = dl4["Sealing"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)       
dl4["Seal"] = dummies       

a = dl4["Escutcheon Plate Installation"].tolist()
print (len(a))
print (len(dl4))
dummies = []
for i in range(0, len(a)):
    if "N" in a[i]:
        dummies.append(1)
    else:
        dummies.append(0)      
dl4["Escutcheon Plate"] = dummies       

print ("DONE")

Create dummies
195072
195072
195072
195072
195072
195072
195072
195072
195072
195072
195072
195072
DONE


### Create Apartment and Public Space Category

In [52]:
locationlist = dl4["location"].tolist()
workspace = []
for i in range(0, len(dl4)):
    a = locationlist[i]
    a = a.split(".")
    #print (a)
    
    if len(a) < 5:
        workspace.append("Public Space")
    else:
        workspace.append("Apartment")
        
print (len(dl4))
print (len(workspace))
    
dl4["Work Space Type"] =   workspace  

195072
195072


### Create Hour Field

In [53]:
#dl4["Hour of Day"] = dl4["Finish Time"].hour

hours = dl4["Finish Time"].tolist()

hourofday = []

for i in range(0, len(hours)):
    #print (hours[i])
    
    a = hours[i].hour
    
    hourofday.append(a)
    
print (len(hourofday))
print (len(dl4))

dl4["Hour of Day"] = hourofday




195072
195072


### Create NYCHA employee field

In [54]:
dp = pd.read_csv(path + "Pest Management Routes as of 1 31 23 CORRECTED.csv")
dp["Name"] = dp["Name"].astype(str).map(str.upper).map(str.strip)
dp["Last Name"] = dp["Name"] .str.split(",").str[0].map(str.strip)
dp["First Name"] = dp["Name"] .str.split(",").str[1].map(str.strip)

dp["Last Name"] = dp["Last Name"].map(str.strip)
dp["First Name"] = dp["First Name"].map(str.strip)

dp["Full Name"] = dp["First Name"] + " " + dp["Last Name"]

dp["NYCHA Employee"] = "NYCHA Employee (1_31_23)"

dp["Full Name"] = dp["Full Name"].map(str.strip)

dp["Full Name"] = dp["Full Name"].str.replace('AGATHA DESVIGNES-MCKAIN', 'AGATHA DESVIGNES MCKAIN')
dp["Full Name"] = dp["Full Name"].str.replace('ALVEN LAKE', 'ALVIN LAKE')
dp["Full Name"] = dp["Full Name"].str.replace('BRIAN JEFFRIES', 'BRIAN JEFFERIES')
dp["Full Name"] = dp["Full Name"].str.replace('CELESTINE R PAYNE', 'CELESTINE PAYNE')
dp["Full Name"] = dp["Full Name"].str.replace('DIEDRA EPHRAIM', 'DEIDRA EPHRAIM')
dp["Full Name"] = dp["Full Name"].str.replace('DIOGENES UMANZAR', 'DIOGENES UMANZOR')
dp["Full Name"] = dp["Full Name"].str.replace('GEOVANI DORSEY', 'GEOVANNIE DORSEY')
dp["Full Name"] = dp["Full Name"].str.replace('HERIBERTO CANTRES', 'HERIBERTO CANTES')
dp["Full Name"] = dp["Full Name"].str.replace('JEMEEL TUCKER', 'JAMEEL TUCKER')
dp["Full Name"] = dp["Full Name"].str.replace('JOSEPH UPSUR', 'JOSEPH UPSHUR')
dp["Full Name"] = dp["Full Name"].str.replace('LAYR DORZIN', 'LAYR DORZIN, JR')
dp["Full Name"] = dp["Full Name"].str.replace('LENAIR DANIELS', 'LENAIR DANIEL')
dp["Full Name"] = dp["Full Name"].str.replace('LORRAINE LOWERY- CLARK', 'LORRAINE LOWERY-CLARK')
dp["Full Name"] = dp["Full Name"].str.replace('MATTEW MARTINEZ', 'MATTHEW MARTINEZ')
dp["Full Name"] = dp["Full Name"].str.replace('NUDUKA OKORO', 'NDUKA OKORO')
dp["Full Name"] = dp["Full Name"].str.replace('OLUYINKA ODUMNBAKU', 'OLUYINKA ODUNMBAKU')
dp["Full Name"] = dp["Full Name"].str.replace('SALVATORE CASTELLENO', 'SALVATORE CASTELLANO')
dp["Full Name"] = dp["Full Name"].str.replace('SHERIF RASHID', 'SHAREEF RASHID')
dp["Full Name"] = dp["Full Name"].str.replace('SHANICE CALLWOOD', 'SHAUNICE CALLWOOD')
dp["Full Name"] = dp["Full Name"].str.replace('SOLOMON SOOKOO', 'SOLOMAN SOOKOO')

dp["Full Name"] = dp["Full Name"].str.replace('VICAS FRANCIS', 'VICKAS FRANCIS')


dpDic = dp.set_index('Full Name')['NYCHA Employee'].to_dict()

dpList = dp["Full Name"].tolist()







In [ ]:
"""

dln = dl4[["Name"]]
dln["Name"] = dln["Name"].astype(str).map(str.upper).map(str.strip)
dln = dln.drop_duplicates(['Name']).reset_index(drop=True)

dln["NYCHA Employee"] = dln["Name"].map(dpDic)

dln = dln.sort_values(by = 'Name', ascending=True).reset_index(drop=True)

dlnList = dln["Name"].tolist()

#dln.to_csv(path + "test_names.csv", index=False)

from fuzzywuzzy import fuzz

from operator import itemgetter


def fuzzy_match(string1, string2):
    
    #Compares two strings using fuzzy matching and returns a score between 0 and 100.

    return fuzz.ratio(string1, string2)

dln["NYCHA Employee"] = dln["NYCHA Employee"].astype(str)
dln["Fuzzy Name Closest Match"] = ""
dln["Fuzzy Score"] = ""

for i in range(0, len(dln)):
    if dln["NYCHA Employee"][i]=='nan' :
        n = dln["Name"][i]
        #print (n)
        
        namematch = []

        for j in range(0, len(dpList)):
            #print (dpList[j], fuzzy_match(n, dpList[j]))
            namematch.append([dpList[j], fuzzy_match(n, dpList[j])])
            
            namematch = sorted(namematch, key=itemgetter(1), reverse=True)

        dln["Fuzzy Name Closest Match"][i] = namematch[0][0]
        dln["Fuzzy Score"][i] = namematch[0][1]


"""

In [ ]:
#dln.to_csv(path + "test_names.csv", index=False)

### Output

In [55]:
all_data2.to_csv(path + "Work Time Analytics.csv", index=False)

In [56]:
qnhours.to_csv(path + "Start and End Time Modes.csv", index=False)

In [57]:
result.to_csv(path + "Time Mode and Employee Names.csv", index=False)

In [58]:
dl4.to_csv(path + "Pest Labor Transactions.csv", index=False)

In [92]:
#dl4.to_csv(path + "test.csv", index=False)